In [1]:
# Daniel loads his data / If we want different data we do that here
# For a encoder decoder model we will need 3 inputs:
# encoder input, decoder support, blend factor (for how much we want to do teacher forcing)
from load_DER_data import load_dataset
raw_data = load_dataset()

from datasets_utils import datasets_from_data
inp, ev, pdf = datasets_from_data(raw_data,
                                  sw_len_samples=int(7*24*(60/5)),
                                  fc_len_samples=int(1*25*(60/5)),
                                  fc_steps=25,
                                  fc_tiles=33,
                                  target_dims=[0,1,2,3,4],
                                  plot=True,
                                 steps_to_new_sample=15)
import numpy as np
ev = np.expand_dims(ev, axis=-1)
inp_train = inp[:int(0.8*inp.shape[0]),:,:]
inp_test = inp[int(0.8*inp.shape[0]):,:,:]
pdf_train = pdf[:int(0.8*inp.shape[0]),1:,:]
pdf_test = pdf[int(0.8*inp.shape[0]):,1:,:]
ev_train = ev[:int(0.8*inp.shape[0]),1:,:]
ev_test = ev[int(0.8*inp.shape[0]):,1:,:]
ev_teacher_train = ev[:int(0.8*inp.shape[0]),:-1,:]
ev_teacher_test = ev[int(0.8*inp.shape[0]):,:-1,:]

blend_factor = np.expand_dims(np.ones(inp_train.shape[0]), axis=-1)
print(blend_factor.shape)



print('The training set has an input data shape of ', 
      inp_train.shape,
      'to expected value targets of ',
     ev_train.shape,
     'or alternatively pdf_targets of ',
     pdf_train.shape)
print('-----------------------------------------------')
print('The testing set has an input data shape of ', 
      inp_test.shape,
      'to expected value targets of ',
     pdf_test.shape,
     'or alternatively pdf_targets of ',
     ev_test.shape)


fetching NWP...
Simulations start at:  2016-01-01 18:00:00
Sample time of Sims is:  0:05:00
Sample time of simulation: 0:05:00
missing  503  samples from  2016-12-31 00:00:00 to  2017-01-01 18:00:00
Setting flags for those datapoints to 1
(649, 12) (503, 13)
missing  1079  samples from  2017-03-23 00:00:00 to  2017-03-26 18:00:00
Setting flags for those datapoints to 1
(649, 12) (1079, 13)
replacing the fauly values with the mean of the array
fetched NWP ... now fetching PV Data
Loading generation data from  2016-01-0117:00:00  to  2017-12-3117:05:00
fetched PV ... now concatenating


c:\users\danielcmaz\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
c:\users\danielcmaz\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


<Figure size 2000x1500 with 17 Axes>

0 percent converted
9 percent converted
19 percent converted
29 percent converted
39 percent converted
49 percent converted
59 percent converted
69 percent converted
79 percent converted
89 percent converted
(10704, 1)
The training set has an input data shape of  (10704, 2016, 16) to expected value targets of  (10704, 24, 1) or alternatively pdf_targets of  (10704, 24, 33)
-----------------------------------------------
The testing set has an input data shape of  (2677, 2016, 16) to expected value targets of  (2677, 24, 33) or alternatively pdf_targets of  (2677, 24, 1)


In [10]:
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session() # make sure we are working clean


# this builds the wrapper class for the multilayer LSTM we will be using for most of the stuff
# in addition to the multiple LSTM layers it adds
# dropout
# layer norm (or maybe recurrent layer norm, whatever feels best)
# inits are the num_layers, num_units per layer and dropout rate
# call needs the inputs and the initial state
# it gives the outputs of all RNN layers (or should it be just one?)
# and the states at the last step
class multi_layer_LSTM(tf.keras.layers.Layer):
    def __init__(self, num_layers, num_units, dropout_rate):
        super(multi_layer_LSTM, self).__init__()
        pass
    
    def call(self, inputs, init_state):
        pass
    

# this builds a basic encoder decoder model with:
# encoder layers / units == decoder layers / units
# output shape is the desired shape of the output [timesteps, dimensions]
# input shape is the supplied encoder input shape [inp_tmesteps, input_dimensions]
# the model will have three inputs:
# the encoder input [inp_tmesteps, input_dimensions]
# the decoder input  [timesteps, dimensions](for teacher forcing and blending to non-teacher forcing)
# the blend factor between decoder model output and real target out
# the model will have one output
# output [timesteps, dimensions]
def build_model(E_D_layers, E_D_units, in_shape, out_shape):
    # Build Encoder Layers 
    encoder_inputs = tf.keras.layers.Input(shape=(inp_train.shape[1], inp_train.shape[2]))
    # ------------------------------------------------------------------------------------------
    layers = E_D_layers # remember units and layers
    units = E_D_units
    encoder = [] # define the encoder layers
    for layer in range(E_D_layers):
        encoder.append(tf.keras.layers.LSTM(units,
                               return_sequences=True,
                               return_state=True,
                               recurrent_initializer='glorot_uniform'))
    # ------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------
    # build Decoder
    decoder_inputs = tf.keras.layers.Input(shape=(out_shape[0], out_shape[1]))
    blend_factor_input = tf.keras.layers.Input(shape=(1))
    # ------------------------------------------------------------------------------------------
    decoder = []
    layers = E_D_layers
    units = E_D_units
    for layer in range(layers):
        decoder.append(tf.keras.layers.LSTM(units,
                               return_sequences=True,
                               return_state=True,
                               recurrent_initializer='glorot_uniform'))

    decoder_wrap = [] #currently it seems that this layer is still creating errors of we have more than one... why?
    if units > 512:
        decoder_wrap.append(tf.keras.layers.Dense(units=256,
                                                 activation='relu'))
    decoder_wrap.append(tf.keras.layers.Dense(units=out_shape[1],
                                             activation='relu'))
    
    concat_layer = tf.keras.layers.Concatenate(axis=1) #needed to concatenate the single timesteps
    reshape_to_1ts = tf.keras.layers.Reshape((1,out_shape[1])) #because keras is wierd with multiplications, this layer is here so keras know all the dimensions
    # ------------------------------------------------------------------------------------------
    # ------------------------------------------------------------------------------------------

    # Run Encoder, we want the encoder states [state_h, state_c] and encoder out
    encoder_states = []
    encoder_outputs = []
    enc_output_layer = None
    for layer in range(layers):
        if layer == 0:
            enc_output_layer, state_h, state_c = encoder[layer](encoder_inputs)
            encoder_states.append([state_h, state_c])
            encoder_outputs.append(enc_output_layer)
        else:
            enc_output_layer, state_h, state_c = encoder[layer](enc_output_layer)
            encoder_states.append([state_h, state_c])
            encoder_outputs.append(enc_output_layer)

    # Run Decoder with Encoder stuffs
    dec_states = encoder_states #encoder states are now decoder states
    blend_factor = reshape_to_1ts(blend_factor_input) #because, otherwise keras will be acting wierd as the decoder_output is (1,1) and this would be (1)

    for t in range(out_shape[0]):
        buffer = reshape_to_1ts(decoder_inputs[:,t,:]) # slice a timestep of the support
        if t == 0: # if no previous output we cannot blend
            dec_t_minus_1 = buffer
        else: # otherwise blend as desired
            dec_t_minus_1 = (1.0-blend_factor)*dec_t_minus_1 + blend_factor*buffer
            
        for layer in range(layers): # feed last timesteps outputs as inputs to the decoder and propagate
            if layer == 0:
                output_layer, state_h, state_c = decoder[layer](dec_t_minus_1, initial_state=dec_states[layer])
                dec_states[layer] = [state_h, state_c] # replace the previous decoder states
            else:
                output_layer, state_h, state_c = decoder[layer](output_layer, initial_state=dec_states[layer])
                dec_states[layer] = [state_h, state_c] # replace the previous decoder states
        
        dec_t_minus_1 = output_layer #go though the decoder projection
        for layer in range(len(decoder_wrap)):
            dec_t_minus_1 = decoder_wrap[layer](dec_t_minus_1)
            
        dec_t_minus_1 = reshape_to_1ts(dec_t_minus_1)
        
        if t == 0: #if no previous we cannot append
            decoder_output = dec_t_minus_1
        else: #else we append
            decoder_output = concat_layer([decoder_output, dec_t_minus_1])
    
    print(decoder_output)
    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = tf.keras.Model([encoder_inputs, decoder_inputs, blend_factor_input], decoder_output)
    return model

In [ ]:
# dumb variation of the encoder decoder
# this will be using teacher forcing
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session() # make sure we are working clean


E_D_layers = 2
E_D_units = 600
out_shape = [ev_train.shape[1], ev_train.shape[2]]
in_shape = [inp_train.shape[1], inp_train.shape[2]]
model = build_model(E_D_layers, E_D_units, in_shape, out_shape) #get a E_D_model

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3) # set optimizers, metrics and loss
loss = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.MeanAbsolutePercentageError()]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics) #compile, print summary
model.summary()

#ToDo: insert loop that adjusts the blend_factor from 1 to 0 depending on how the validation loss develops
for epoch in range(20):
    history = model.fit(x=[inp_train, ev_teacher_train, blend_factor], #train for a given set of epochs, look at history
              y=ev_train,
              batch_size=32,
              epochs=1,
              validation_split=0.2)
    val_metric = history.history['val_mean_absolute_percentage_error']
    if epoch == 0:
        val_metric_before = val_metric
    
    if val_metric > val_metric_before:
        blend_factor = 0.9*blend_factor
    
    val_metric_before = val_metric

Tensor("concatenate_22/Identity:0", shape=(None, 24, 1), dtype=float32)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 24, 1)]      0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [None]               0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2016, 16)]   0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 1, 1)         0           input_3[0][0]                    
                      

Train on 8563 samples, validate on 2141 samples
8563/8563 [==============================] - 176s 21ms/sample - loss: 0.3098 - mean_absolute_percentage_error: 157.0218 - val_loss: 0.2564 - val_mean_absolute_percentage_error: 288.6730
Train on 8563 samples, validate on 2141 samples
8512/8563 [============================>.] - ETA: 0s - loss: 0.2711 - mean_absolute_percentage_error: 138.2425